<a href="https://colab.research.google.com/github/suresh-venkate/Code_Repositories/blob/main/Deep_Learning/Computer_Vision/Projects/Car_BBOX_Detection/TF_ObjDet_SSD_MobNet/Car_BBOX_Det_SSD_Mob_Net_Infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Bounding Box Detection - SSD MobileNet Detector - Inference NB


# Copy model and complete other preliminaries

## Connect to Google Drive

In [ ]:
  from google.colab import drive
  drive.mount('/content/drive')

## Copy required files

In [ ]:
%%capture

# Copy dataset from Google Drive
!cp /content/drive/MyDrive/AI_ML_Folder/Colab_Directory/Datasets/Image_Datasets/Stanford_Car_Dataset/Consolidated_Dataset.zip .
# Unzip dataset file
!unzip Consolidated_Dataset.zip
# Delete zip file
!rm Consolidated_Dataset.zip

## Copy model and label class mapping from Google Drive

In [ ]:
# Copy model directory
!cp -r /content/drive/MyDrive/AI_ML_Folder/Colab_Directory/Model_Outputs/Car_BBOX_SSD_MobNet_Model/Trained_Models/M1/detection_model .
# Copy label-class mapping pkl file
!cp /content/drive/MyDrive/AI_ML_Folder/Colab_Directory/Model_Outputs/Car_BBOX_SSD_MobNet_Model/Trained_Models/M1/car_dataset_label.pkl .

## Import required libraries

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import datetime
import pytz
import time
import os
import cv2
import tensorflow as tf

from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline

## Extract label_class_dict from pkl file

In [ ]:
# Pickle file where index to label mapping is stored
fname = "/content/car_dataset_label.pkl" 
with open(fname, "rb") as file:
    label_class_dict = pickle.load(file)
    
print(f"Type of label_class_dict is {type(label_class_dict)}")
print()
print(f"Keys of label_class_dict are:")
print("--------------------------------")
print(label_class_dict.keys())
print()
print("First five elements of label_class_dict are:")
print("-----------------------------------------------")
for ind in range(1, 6, 1):
    print(label_class_dict[ind])

## Load saved model

In [ ]:
%%capture
# Define path where model is stored
model_path = '/content/detection_model/saved_model'
# Load model from path
model = tf.saved_model.load(model_path)

## Create train_df and test_df

In [ ]:
# Load train annotation file in a DataFrame
ann_train_csv_path = '/content/annot_train_cons.csv'
train_df = pd.read_csv(ann_train_csv_path)
display(train_df.head())

In [ ]:
# Load test annotation file in a DataFrame
ann_test_csv_path = '/content/annot_test_cons.csv'
test_df = pd.read_csv(ann_test_csv_path)
display(test_df.head())

In [ ]:
# Get dataset size
print("Number of images in training set is %d" %train_df.shape[0])
print("Number of images in test set is %d" %test_df.shape[0])

# Make predictions

## Function: predict_and_plot

In [ ]:
def predict_and_plot(df, samp_indices, img_root_path, model, num_cols, col_size, row_size, fig_title,\
                     box_thickness):

  '''
  Function to make predictions on a subset of data and plot images with actual
  and predicted labels

  Arguments:
    X: Input image array
    y: Input label array
    samp_indices: Indices from X to be plotted
    model: Model instance to use for making predictions
    num_cols: Number of columns to use for plotting
    col_size: Size of columns to use for plotting
    row_size: Size of rows to use for plotting
    fig_title: Title to use for overall figure
  ''' 

  num_rows = math.ceil(len(samp_indices) / num_cols) # Number of rows to use for plotting
  fig = plt.figure(figsize = ((num_cols * col_size), (num_rows * row_size)))
  fig.suptitle(fig_title, fontsize = 40)

  for ind, samp_ind in enumerate(samp_indices): # Loop through samp_indices
    img_file_name = df.loc[samp_ind, 'filename'] # Extract image filename from DataFrame
    img_file_path = os.path.join(img_root_path, img_file_name) # Obtain image file path
    img = cv2.cvtColor(cv2.imread(img_file_path), cv2.COLOR_BGR2RGB) # Load image
    img_mod = np.expand_dims(img, axis = 0) # Add batch axis to feed to model
    model_out = model(img_mod) # Pass input image through model

    xmin, ymin, xmax, ymax = df.loc[samp_ind, ['xmin', 'ymin', 'xmax', 'ymax']] # Get ground-truth 
                                                                                # BBOX co-ords
    img_h, img_w = df.loc[samp_ind, ['img_h', 'img_w']] # Extract image dimensions
    act_class = df.loc[samp_ind, 'class'] # Get ground-truth class

    score = 100 * model_out['detection_scores'][0].numpy()[0] # Extract best box score
    bbox = model_out['detection_boxes'][0].numpy()[0] # Extract BBOX co-ordinates
    label = model_out['detection_classes'][0].numpy().astype('int32')[0] # Extract class index
    pred_class = label_class_dict[label] # Obtain class of car

    # Scale BBOX co-ordinates to image dimensions
    (bbox_ymin, bbox_xmin, bbox_ymax, bbox_xmax) = (int(bbox[0] * img_h), int(bbox[1] * img_w),\
                                                    int(bbox[2] * img_h), int(bbox[3] * img_w))  
    
    # Draw ground-truth BBOX in green
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), box_thickness)
    # Draw predicted BBOX in blue    
    cv2.rectangle(img, (bbox_xmin, bbox_ymin), (bbox_xmax, bbox_ymax), (0, 0, 255), box_thickness)

    ax = plt.subplot(num_rows, num_cols, (ind + 1))
    ax.text(0, -40, f"Actual class: {act_class}", fontsize = 'xx-large')
    ax.text(0, -10, f"Predicted class: {pred_class} ({score:0.2f}%)", fontsize = 'xx-large')
    ax.imshow(img) 

## Training Set Predictions

In [ ]:
train_img_root_path = "/content/train_images"
fig_title = 'Random Images from Training Set'
num_samp_to_plot = 8
samp_indices = list(np.random.randint(low = 0, high = train_df.shape[0], size = num_samp_to_plot))

predict_and_plot(train_df, samp_indices, train_img_root_path, model, 2, 8, 8, fig_title, 2)

## Test Set Predictions

In [ ]:
test_img_root_path = "/content/test_images"
fig_title = 'Random Images from Testing Set'
num_samp_to_plot = 8
samp_indices = list(np.random.randint(low = 0, high = test_df.shape[0], size = num_samp_to_plot))
predict_and_plot(test_df, samp_indices, test_img_root_path, model, 2, 8, 8, fig_title, 2)

# Evaluate Performance

## Function: iou computation

In [ ]:
def iou(box1, box2):
    
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """

    # Extract x, y co-ordinates of both boxes
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (xi1, yi1, xi2, yi2) coordinates of the intersection of box1 and box2. 
    # Calculate its Area.
    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)
    inter_width = max((xi2 - xi1),0)
    inter_height = max((yi2 - yi1),0)
    inter_area = inter_width * inter_height

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1_y2 - box1_y1) * (box1_x2 - box1_x1)
    box2_area = (box2_y2 - box2_y1) * (box2_x2 - box2_x1)
    union_area = box1_area + box2_area - inter_area
    
    # compute the IoU
    iou = inter_area / union_area
    
    return iou

##Function: model_eval

In [ ]:
def model_eval(df, img_root_path, num_samp):

  '''
  Function to evaluate the performance of the model given a certain set of images. 
  Classificaton accuracy and Mean IoU over all images will be computed and reported

  Arguments:
    df: Input Dataframe with image information
    img_root_path: Root path of images
    num_samp: Number of samples to use for evaluating model
  '''
  
  # Create a dataframe placeholder to store evaluation results
  df_cols = ['Act_Class', 'Pred_Class', 'Act_xmin', 'Act_ymin', 'Act_xmax', 'Act_ymax',\
           'Pred_xmin', 'Pred_ymin', 'Pred_xmax', 'Pred_ymax']
  results_df = pd.DataFrame(columns = df_cols) # Placeholder to store results
  corr_pred = 0 # counter to keep track of correct predictions
  iou_list = [] # List to store IoU values (of Actual BBOX vs. Predicted BBOX) for each image

  for samp_ind in tqdm(range(num_samp)): # Loop through num_samp
    img_file_name = df.loc[samp_ind, 'filename'] # Extract image filename from DataFrame
    img_file_path = os.path.join(img_root_path, img_file_name) # Obtain image file path
    img = cv2.cvtColor(cv2.imread(img_file_path), cv2.COLOR_BGR2RGB) # Load image
    img_mod = np.expand_dims(img, axis = 0) # Add batch axis to feed to model
    model_out = model(img_mod) # Pass input image through model 

    act_class = df.loc[samp_ind, 'class'] # Get ground-truth class
    label = model_out['detection_classes'][0].numpy().astype('int32')[0] # Extract class index
    pred_class = label_class_dict[label] # Obtain class prediction
    bbox = model_out['detection_boxes'][0].numpy()[0] # Extract BBOX co-ordinates
    img_h, img_w = df.loc[samp_ind, ['img_h', 'img_w']] # Extract image dimensions

    # Create entry_dict for updating dataframe
    entry_dict = {'Act_Class': act_class,
                  'Pred_Class': pred_class,
                  'Act_xmin': df.loc[samp_ind, 'xmin'],
                  'Act_ymin': df.loc[samp_ind, 'ymin'],
                  'Act_xmax': df.loc[samp_ind, 'xmax'],
                  'Act_ymax': df.loc[samp_ind, 'ymax'],
                  'Pred_xmin': int(bbox[1] * img_w),
                  'Pred_ymin': int(bbox[0] * img_h),
                  'Pred_xmax': int(bbox[3] * img_w),
                  'Pred_ymax': int(bbox[2] * img_h),
                }
    results_df = results_df.append(entry_dict, ignore_index = True) # Update dataframe

    # Update corr_pred
    corr_pred += int(entry_dict['Act_Class'] == entry_dict['Pred_Class'])

    # Compute IoU
    box1 = (entry_dict['Act_xmin'], entry_dict['Act_ymin'],\
            entry_dict['Act_xmax'], entry_dict['Act_ymax'])
    box2 = (entry_dict['Pred_xmin'], entry_dict['Pred_ymin'],\
            entry_dict['Pred_xmax'], entry_dict['Pred_ymax'])
    iou_list.append(iou(box1, box2))

  tot_pred = results_df.shape[0]
  print()
  print()
  print("Classification accuracy is %0.2f %%" %(100 * (corr_pred / tot_pred)))
  print("Mean IoU score is %0.4f" %(np.mean(iou_list)))

  return results_df, iou_list

## Evaluate model on training set

In [ ]:
train_img_root_path = "/content/train_images"
train_results_df, train_iou_list = model_eval(train_df, train_img_root_path, 100)

## Evaluate model on test set

In [ ]:
test_img_root_path = "/content/test_images"
test_results_df, test_iou_list = model_eval(test_df, test_img_root_path, 100)